<a href="https://colab.research.google.com/github/ssrbazpur/BigQuery/blob/master/Model_Building_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/nyc-taxi-265120-baf9a3e4cf9b.json"
%reload_ext google.cloud.bigquery

In [0]:

project_id = 'nyc-taxi-265120' #@param{type:"string"}
from google.cloud import bigquery
#from google.cloud import bigquery_storage_v1beta1
# Create BigQuery client
client = bigquery.Client(project = project_id)
# Create BigQuery storage client
#bq_storage_client = bigquery_storage_v1beta1.BigQueryStorageClient()


<h3> Collecting Data Prepared </h3>

In [0]:
#2018 DATA
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
        MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2018`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  
  -- TAXI DATA
  ),
TD AS(

SELECT  zone_id, time, pickups ,CAST(EXTRACT (YEAR from time) AS STRING) AS year,EXTRACT (DAYOFYEAR from time) AS daynumber,EXTRACT (HOUR from time) AS hour from `nyc-taxi-265120.hello.2018Data` ),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)


SELECT 
points.zone_id,
td.time,
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
    wd.mintemp, 
    wd.maxtemp,
    wd.rain,
 td.pickups,

    ST_ASTEXT(points.p) AS P,
 
  FROM wd, td INNER JOIN points ON points.zone_id=td.zone_id
  where wd.year = td.year AND
  wd.daynumber = td.daynumber
  group by zone_id,time,daynumber,hour,pickups,P,dayofweek, mintemp, maxtemp, rain

"""
df1 = client.query(query).to_dataframe()


In [11]:
#2019 DATA
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
        MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2019`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  
  -- TAXI DATA
  ),
TD AS(

SELECT  zone_id, time, pickups ,CAST(EXTRACT (YEAR from time) AS STRING) AS year,EXTRACT (DAYOFYEAR from time) AS daynumber,EXTRACT (HOUR from time) AS hour from `nyc-taxi-265120.hello.test_table` ),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)


SELECT 
points.zone_id,
td.time,
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
    wd.mintemp, 
    wd.maxtemp,
    wd.rain,
 td.pickups,

    ST_ASTEXT(points.p) AS P,
 
  FROM wd, td INNER JOIN points ON points.zone_id=td.zone_id
  where wd.year = td.year AND
  wd.daynumber = td.daynumber
  group by zone_id,time,daynumber,hour,pickups,P,dayofweek, mintemp, maxtemp, rain

"""
df2 = client.query(query).to_dataframe()


KeyboardInterrupt: ignored

In [0]:
#DROP p
df1.drop('P', axis=1, inplace=True)
df2.drop('P', axis=1, inplace=True)

In [13]:
df1.head()

,zone_id,time,daynumber,hour,dayofweek,mintemp,maxtemp,rain,pickups,P
0,1,2018-01-01 00:00:00+00:00,1,0,2,8.1,21.9,0.0,0.0,POINT(-74.1740015721158 40.6918301401778)
1,1,2018-01-01 01:00:00+00:00,1,1,2,8.1,21.9,0.0,0.0,POINT(-74.1740015721158 40.6918301401778)
2,1,2018-01-01 02:00:00+00:00,1,2,2,8.1,21.9,0.0,0.0,POINT(-74.1740015721158 40.6918301401778)
3,1,2018-01-01 05:00:00+00:00,1,5,2,8.1,21.9,0.0,0.0,POINT(-74.1740015721158 40.6918301401778)
4,1,2018-01-01 19:00:00+00:00,1,19,2,8.1,21.9,0.0,0.0,POINT(-74.1740015721158 40.6918301401778)
